# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [40]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [34]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop('income', axis = 1)
Y = adult_dt['income']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y ,test_size = 0.3 , random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*
Random state is used to generate the same observations for train and test splits so that when we run the code again, we get the same combination of observations in these sets. This helps reproduce the same results if the input is the same dataset. Consequently, the output remains consistent, which allows us to compare different models using the same random_state. This ensures that the data splits are the same across different runs, facilitating fair and accurate comparisons.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer

numerical_columns = adult_dt.select_dtypes(include = ['int', 'float']).columns.to_list()
categorical_columns = adult_dt.select_dtypes(include=['object']).columns.to_list()
numerical_columns.remove('income')  

numerical_pipeline = Pipeline([('KNNimputer', KNNImputer(n_neighbors=7, weights='distance')),
                               ('Robustscaler', RobustScaler())])

categorical_pipeline = Pipeline([('SimpleImputer', SimpleImputer(strategy= 'most_frequent')),
                                 ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'))])

columnTransformer = ColumnTransformer([('num_transforms', numerical_pipeline, numerical_columns),
                                      ('cat_transforms', categorical_pipeline, categorical_columns)])
print(categorical_columns)
print(numerical_columns)

columnTransformer

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNimputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('Robustscaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [61]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([('preprocessing',columnTransformer),
                     ('classifier', RandomForestClassifier())])
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNimputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('Robustscaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [63]:

from sklearn.model_selection import cross_validate

crossValidate = cross_validate(pipeline, X_train, Y_train, cv=5, scoring=['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy'], return_train_score = True)

crossValidate

{'fit_time': array([7.95284677, 7.88785791, 7.8698256 , 7.88361311, 7.82617474]),
 'score_time': array([0.15101075, 0.16098166, 0.15348887, 0.16605377, 0.15425277]),
 'test_neg_log_loss': array([-0.34276806, -0.34777594, -0.37648756, -0.34882187, -0.38801966]),
 'train_neg_log_loss': array([-0.08169495, -0.08098178, -0.08153688, -0.0823141 , -0.08154145]),
 'test_roc_auc': array([0.90506316, 0.90211908, 0.90142233, 0.90779124, 0.90155128]),
 'train_roc_auc': array([0.99999999, 1.        , 1.        , 1.        , 1.        ]),
 'test_accuracy': array([0.84799298, 0.84689625, 0.85146994, 0.8569548 , 0.85805178]),
 'train_accuracy': array([0.99994515, 1.        , 1.        , 1.        , 0.99994516]),
 'test_balanced_accuracy': array([0.77090302, 0.76833534, 0.77304842, 0.78036071, 0.77800784]),
 'train_balanced_accuracy': array([0.9998869 , 1.        , 1.        , 1.        , 0.99988693])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [67]:
pd_results = pd.DataFrame(crossValidate)
# pd_results.head()
pd_results_sorted = pd_results.sort_values(by = 'test_neg_log_loss',ascending=True)
pd_results_sorted.head()

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
4,7.826175,0.154253,-0.388020,-0.081541,0.901551,1.0,0.858052,0.999945,0.778008,0.999887
2,7.869826,0.153489,-0.376488,-0.081537,0.901422,1.0,0.851470,1.000000,0.773048,1.000000
3,7.883613,0.166054,-0.348822,-0.082314,0.907791,1.0,0.856955,1.000000,0.780361,1.000000
1,7.887858,0.160982,-0.347776,-0.080982,0.902119,1.0,0.846896,1.000000,0.768335,1.000000
0,7.952847,0.151011,-0.342768,-0.081695,0.905063,1.0,0.847993,0.999945,0.770903,0.999887


Calculate the mean of each metric. 

In [68]:
pd.DataFrame(crossValidate).mean()


fit_time                   7.884064
score_time                 0.157158
test_neg_log_loss         -0.360775
train_neg_log_loss        -0.081614
test_roc_auc               0.903589
train_roc_auc              1.000000
test_accuracy              0.852273
train_accuracy             0.999978
test_balanced_accuracy     0.774131
train_balanced_accuracy    0.999955
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [74]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

pipeline.fit(X_train, Y_train)

Y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
Y_pred = pipeline.predict(X_test)

neg_log_loss = log_loss(Y_test, Y_pred_proba)
roc_auc = roc_auc_score(Y_test, Y_pred_proba)

accuracy = accuracy_score(Y_test, Y_pred)
balanced_accuracy = balanced_accuracy_score(Y_test, Y_pred)
# print(neg_log_loss, roc_auc, accuracy, balanced_accuracy)

result_dic = {'accuracy': accuracy,
              'balanced_accuracy': balanced_accuracy,
              'neg_log_loss': neg_log_loss,
              'roc_auc': roc_auc}
result_dic

{'accuracy': 0.8538233186610708,
 'balanced_accuracy': 0.77280119785263,
 'neg_log_loss': 0.3759036090636723,
 'roc_auc': 0.8993436852750035}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.